# Productividad vtr

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta  # Asegúrate de importar timedelta


In [2]:
import pandas as pd

# Ruta del archivo
ruta = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\soportepostventa_Attendants_Status_2025-07-01_2025-07-30.xlsx"

# Leer el archivo Excel (por defecto lee la primera hoja)
prod = pd.read_excel(ruta)

In [3]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
prod['MAIL'] = prod['AgentIdentity'].str.split('@').str[0]

In [5]:
# Lista de columnas a convertir
cols_to_int = [
    "Online", "Invisible", "Pause", "Offline",
    "OpenTicketOnline", "OpenTicketInvisible", "WithoutTicketOnline"
]

# Reemplazar nulos por 0 y convertir a entero
for col in cols_to_int:
    prod[col] = prod[col].fillna(0).astype(int)


In [6]:
# Convertir el campo a datetime
prod["DateTimeRef"] = pd.to_datetime(prod["DateTimeRef"], dayfirst=True, errors="coerce")

# Crear campo Fecha en formato dd-mm-aaaa
prod["Fecha"] = prod["DateTimeRef"].dt.strftime("%d-%m-%Y")

# Crear campo INTERVALO con hora en formato HH:MM
prod["INTERVALO"] = prod["DateTimeRef"].dt.strftime("%H:%M")


In [7]:
prod.sample(1)

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO
32783,herlly.floresmedina@e.vtr.cl,soportepostventa@msging.net,"[Tecnico_Fijo, soporte_ott, Masivos, DBOX SMAR...",Web,2025-07-03 23:00:00,59,0,0,0,55,0,4,0.0,herlly.floresmedina,03-07-2025,23:00


In [8]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\wduran\AppData\Local\Temp\ipykernel_3988\3964556920.py:3: DtypeWarning: Columns (17,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,101) have mixed types. Specify dtype option on import or set low_memory=False.
  blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'


In [10]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\VTR\Dotación_VTR_20250725.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacion = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [11]:
dotacion_filtrada = dotacion[
    (dotacion['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacion['PLATAFORMA'].str.lower().isin(['0 wsp. oopp', '0 wsp postventa'])) &
    (dotacion['CARGO'].str.lower() == 'asesor') &
    (dotacion['ESTADO_ASESOR'].str.lower().isin(['activo', 'licencia', 'vacaciones']))
]


In [13]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]

C:\Users\wduran\AppData\Local\Temp\ipykernel_3988\3410713443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]


In [14]:
# Creamos versiones en minúscula del campo MAIL para el join
prod['MAIL'] = prod['MAIL'].str.lower()
dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
blip_ha['MAIL'] = blip_ha['MAIL'].str.lower()

C:\Users\wduran\AppData\Local\Temp\ipykernel_3988\1444816432.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()


In [16]:
# Hacer merge directo por MAIL
prod = prod.merge(
    dotacion_filtrada[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
prod['MARCA'] = prod['EMPRESA_PAIS'].notna().map({True: 'VTR', False: ''})


In [18]:
# Asegurarnos de que DateTimeRef es datetime
prod["DateTimeRef"] = pd.to_datetime(prod["DateTimeRef"], dayfirst=True, errors="coerce")

# Reconvertir campo Fecha a formato YYYY-MM-DD
prod["Fecha"] = prod["DateTimeRef"].dt.strftime("%Y-%m-%d")


In [19]:
# --- 3) Contar SequentialId por MAIL-Fecha-INTERVALO en blip_ha ---
blip_counts = (
    blip_ha
    .groupby(["MAIL", "Fecha", "INTERVALO"], as_index=False)["SequentialId"]
    .count()
    .rename(columns={"SequentialId": "Atenciones"})
)

# --- 4) Merge a prod para traer Atenciones (dejar 0 si no hay match) ---
prod = prod.merge(
    blip_counts,
    on=["MAIL", "Fecha", "INTERVALO"],
    how="left"
)

prod["Atenciones"] = prod["Atenciones"].fillna(0).astype(int)


In [20]:
prod.sample(4)

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO,EMPRESA_PAIS,MARCA,Atenciones
17522,danae.medrano@e.vtr.cl,soportepostventa@msging.net,"[Comercial_Fijo, consulta_boleta, OP, OP_Def];...",Web,2025-07-20 02:00:00,0,0,0,59,0,0,0,0.0,danae.medrano,2025-07-20,02:00,KONECTA PERU,VTR,0
31833,hector.prietobellido@e.vtr.cl,soportepostventa@msging.net,"[Tecnico_Fijo, soporte_ott, Masivos, DBOX SMAR...",Web,2025-07-21 09:00:00,0,0,0,59,0,0,0,0.0,hector.prietobellido,2025-07-21,09:00,KONECTA PERU,VTR,0
73438,nathalia.grajales@e.vtr.cl,soportepostventa@msging.net,"[Cambio_Dirección, red_neutra_out, RedNeutra_v...",Web;Mobile,2025-07-15 22:00:00,0,0,0,59,0,0,0,0.0,nathalia.grajales,2025-07-15,22:00,ATENTO COLOMBIA,VTR,0
80153,rosa.otinianochavarra@e.vtr.cl,soportepostventa@msging.net,[],Web,2025-07-04 17:00:00,59,0,0,0,0,0,59,0.0,rosa.otinianochavarra,2025-07-04,17:00,KONECTA PERU,VTR,0


In [21]:
# Opcional si Online pudiese venir como texto
prod['H-Productiva'] = (prod['Online'] >= 10).astype(int)

In [22]:
# Eliminar filas con NaN en EMPRESA_PAIS
prod = prod[prod['EMPRESA_PAIS'].notna()].copy()


In [24]:
# Ahora sí, guardás el archivo
prod.to_excel(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\Productividad_JUL.xlsx", index=False)